# ARMA Models

## Intro to time series and stationarity


#### Exploration

In [ ]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd

# Load in the time series
candy = pd.read_csv('candy_production.csv', 
            index_col='date',
            parse_dates=True)

# Plot and show the time series on axis ax1
fig, ax1 = plt.subplots()
candy.plot(ax=ax1)
plt.show()

#### Train-test split

In [ ]:
# Split the data into a train and test set
candy_train = candy.loc[:'2006']
candy_test = candy.loc['2007':]

# Create an axis
fig, ax = plt.subplots()

# Plot the train and test sets on the axis ax
candy_train.plot(ax=ax)
candy_test.plot(ax=ax)
plt.show()

### Making time series stationary


#### Testing for stationarity wit Augmented Dicky-Fuller


In [ ]:
# Import augmented dicky-fuller test function
from statsmodels.tsa.stattools import adfuller

# Run test
result = adfuller(earthquake['earthquakes_per_year'])

# Print test statistic
print(result[0])

# Print p-value
print(result[1])

# Print critical values
print(result[4]) 

#### Taking the difference
* .diff() applyed twice

#### Other tranforms

Differencing should be the first transform you try to make a time series stationary. But sometimes it isn't the best option.

A classic way of transforming stock time series is the log-return of the series. This is calculated as follows:

\begin{equation}
log\_return(y_t) = log\frac{y_{t}}{y_{t-1}}
\end{equation}

The Amazon stock time series has already been loaded for you as amazon. You can calculate the log-return of this DataFrame by substituting:

 * y_t -> amazon
 * y_(t-1) -> amazon.shift(1)
 * log -> np.log()

In this exercise you will compare the log-return transform and the first order difference of the Amazon stock time series to find which is better for making the time series stationary.

In [ ]:
# Calculate the first difference and drop the nans
amazon_diff = amazon.diff()
amazon_diff = amazon_diff.dropna()

# Run test and print
result_diff = adfuller(amazon_diff['close'])
print(result_diff)

# Calculate log-return and drop nans
amazon_log = np.log(amazon.div(amazon.shift(1)))
amazon_log = amazon_log.dropna()

# Run test and print
result_log = adfuller(amazon_log['close'])
print(result_log)

## Structure of ARIMA models

#### AR Models

#### MA Models

#### ARMA Models

#### ARMAX Models

* X - Exogenous variable
* ARMAX = ARMA + linear regression

#### Generating ARMA data


In [ ]:
# Import data generation function and set random seed
from statsmodels.tsa.arima_process import arma_generate_sample
np.random.seed(3)

# Set coefficients
ar_coefs = [1, 0.2] # Negative coeficients
ma_coefs = [1, 0.3, 0.4]

# Generate data
y = arma_generate_sample(ar_coefs, ma_coefs, nsample=100, scale=0.5)

plt.plot(y)
plt.ylabel(r'$y_t$')
plt.xlabel(r'$t$')
plt.show()

#### Fitting Prelude


In [ ]:
# Import the ARMA model
from statsmodels.tsa.arima_model import ARMA

# Instantiate the ARMA model 
model = ARMA(y, order=(1,1))

# Instantiate the ARMAX model
model = ARMA(S1, order = (2,1), exog = S2)

# Fit the model
results = model.fit()

# Print summary
print(results.summary())

# Fitting the Future

#### Generating one-step-ahead predictions


In [ ]:
# Generate predictions
one_step_forecast = results.get_prediction(start=-30)

# Extract prediction mean
mean_forecast = one_step_forecast.predicted_mean

# Get confidence intervals of predictions
confidence_intervals = one_step_forecast.conf_int()

# Select lower and upper confidence limits
lower_limits = confidence_intervals.loc[:,'lower close']
upper_limits = confidence_intervals.loc[:,'upper close']

# Print best estimate predictions
print(mean_forecast)

#### Plotting one-step-ahead predictions


In [ ]:
# plot the amazon data
plt.plot(amazon.index, amazon, label='observed')

# plot your mean predictions
plt.plot(mean_forecast.index, mean_forecast, color='r', label='forecast')

# shade the area between your confidence limits
plt.fill_between(lower_limits.index, lower_limits, 
               upper_limits, color='pink')

# set labels, legends and show plot
plt.xlabel('Date')
plt.ylabel('Amazon Stock Price - Close USD')
plt.legend()
plt.show()

#### Generating dynamic forecasts


In [ ]:
# Generate predictions
dynamic_forecast = results.get_prediction(start = -30, dynamic=True)

# Extract prediction mean
mean_forecast = dynamic_forecast.predicted_mean

# Get confidence intervals of predictions
confidence_intervals = dynamic_forecast.conf_int()

# Select lower and upper confidence limits
lower_limits = confidence_intervals.loc[:,'lower close']
upper_limits = confidence_intervals.loc[:,'upper close']

# Print best estimate predictions
print(mean_forecast)

#### Differencing and fitting ARMA

In [ ]:
# Take the first difference of the data
amazon_diff = amazon.diff().dropna()

# Create ARMA(2,2) model
arma = SARIMAX(amazon_diff, order = (2,0,2))

# Fit model
arma_results = arma.fit()

# Print fit summary
print(arma_results.summary())

#### Unrolling ARMA forecast

In [ ]:
# Make arma forecast of next 10 differences
arma_diff_forecast = arma_results.get_forecast(steps=10).predicted_mean

# Integrate the difference forecast
arma_int_forecast = np.cumsum(arma_diff_forecast)

# Make absolute value forecast
arma_value_forecast = arma_int_forecast + amazon.iloc[-1,0]

# Print forecast
print(arma_value_forecast)

#### Using ARIMA

In [ ]:
# Create ARIMA(2,1,2) model
arima = SARIMAX(amazon, order = (2,1,2))

# Fit ARIMA model
arima_results = arima.fit()

# Make ARIMA forecast of next 10 values
arima_value_forecast = arima_results.get_forecast(steps=10).predicted_mean

# Print forecast
print(arima_value_forecast)

# The Best of the Best Models

## ACF & PCF (auto correlation and parcial auto correlation)

* IF (ACF tails off) and (PACF cuts off after lag p): AR(p)
* IF (ACF cuts off after q) and (APCF tails off): MA(q)
* IF both tail: ARMA
* IF (ACF values are high) and (tail slowly): non stationary
* IF (ACF lag-1 = too negative): you took the difference too many times

## AIC & BIC

* AIC: Akaike information criteria
* BIC: Bayesian information criteria

Lower AIC and BIC indicate better models. They penalize excess of parameters.

BIC focusses more on restricting model size and is better for choosing a explanatory model.

AIC is better at choosing predictive models.

#### Searching over model order


In [ ]:
# Create empty list to store search results
order_aic_bic=[]

# Loop over p values from 0-2
for p in range(3):
  # Loop over q values from 0-2
    for q in range(3):
      	# create and fit ARMA(p,q) model
        model = SARIMAX(df, order=(p,0,q))
        results = model.fit()
        
        # Append order and results tuple
        order_aic_bic.append((p,q,results.aic, results.bic)),
        
# Construct DataFrame from order_aic_bic
order_df = pd.DataFrame(order_aic_bic, 
                        columns=['p', 'q', 'AIC', 'BIC'])

# Print order_df in order of increasing AIC
print(order_df.sort_values('AIC'))

# Print order_df in order of increasing BIC
print(order_df.sort_values('BIC'))

## Model Diagnostics

#### Mean absolute error


In [ ]:
# Fit model
model = SARIMAX(earthquake, order=(1,0,1))
results = model.fit()

# Calculate the mean absolute error from residuals
mae = np.mean(np.abs(results.resid))

# Print mean absolute error
print(mae)

# Make plot of time series for comparison
earthquake.plot()
plt.show()

#### Diagnostic summary statistics

It is important to know when you need to go back to the drawing board in model design. In this exercise you will use the residual test statistics in the results summary to decide whether a model is a good fit to a time series.

Here is a reminder of the tests in the model summary:

| Test | Null hypothesis  | P-value name  | IF|
|---|---|---|---|
|  Ljung-Box |  There are no correlations in the residual |  Prob(Q) | p<0.05 : correlated|
|  Jarque-Bera |  The residuals are normally distributed | Prob(JB)  | p<0.05 : non normal|

We reject the null hypothesis if the p-value < 0.05.
		

In [ ]:
# Create and fit model
model1 = SARIMAX(df, order=(3,0,1))
results1 = model1.fit()

# Print summary
print(results1.summary())

#### Plot diagnostics

|Test	| Good fit |
|---|---|
|Standardized residual |	There are no obvious patterns in the residuals|
|Histogram plus kde estimate | The KDE curve should be very similar to the normal distribution|
|Normal Q-Q	 | Most of the data points should lie on the straight line|
|Correlogram	| 95% of correlations for lag greater than zero should not be significant|

In [ ]:
# Create and fit model
model = SARIMAX(df, order=(1,1,1))
results = model.fit()

# Create the 4 diagostics plots
results.plot_diagnostics()
plt.show()

### Box-Jenkins method

From Raw data -> production model

* Identification
* Estimation
* Model siagnostics

#### Identification

* Is the time series stationary?
* What differencing will make it stationary?
* What transforms will make it stationary?
* What values of P and q are most promissing?
    * Plot the time series
    * Use audmented Dicky-fuller test
    * Use transforms .diff(), np.log(), np.sqrt()
    * Plot ACF/PACF and pick a model order.

#### Estimation

* Use data to train model coefficients
* Choose between models using AIC and BIC

#### Model Diagnostics

* Are residuals uncorrelated?
* Are residuals normally distributed?
    * results.plot_diagnostics()
    * results.summary()

#### Is model ok?
* Back to identification
* Send to production

#### Identification

In [ ]:
# Plot time series
savings.plot()
plt.show()

# Run Dicky-Fuller test
result = adfuller(savings.savings)

# Print test statistic
print(result[0])

# Print p-value
print(result[1])

# Create figure
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,8))
 
# Plot the ACF of savings on ax1
plot_acf(savings, lags = 10, zero = False, ax = ax1)

# Plot the PACF of savings on ax2
ax2 = plot_pacf(savings, lags = 10,  zero = False, ax = ax2)

plt.show()

#### Estimation

In [ ]:
# Loop over p values from 0-3
for p in range(4):
  
  # Loop over q values from 0-3
    for q in range(4):
      try:
        # Create and fit ARMA(p,q) model
        model = SARIMAX(savings, order=(p,0,q), trend='c')
        results = model.fit()
        
        # Print p, q, AIC, BIC
        print(p, q, results.aic, results.bic)
        
      except:
        print(p, q, None, None)

#### Diagnostics

In [ ]:
# Create and fit model
model = SARIMAX(savings, order = (1, 0, 2), trend = 'c')
results = model.fit()

# Create the 4 diagostics plots
results.plot_diagnostics()
plt.show()

# Print summary
print(results.summary())

# Seasonal ARIMA Models